# MLlib

Spark biedt ook een framework aan voor MachineLearning modellen te trainen op gedistribueerde datasets.
Dit framework is MLlib of ook wel sparkML genoemd.
De code om te werken met deze package is sterk gelijkaardig aan sklearn.
De API en een uitgebreide documentatie met voorbeeldcode kan je [hier](https://spark.apache.org/docs/latest/ml-guide.html) vinden.

Deze package bied de volgende tools aan
* ML-technieken: classificatie, regressie, clustering, ...
* Features: Extracting en transforming van features, PCA, ...
* Pipelines: Maak, train, optimaliseer en evalueer pipelines
* Persistentie: Bewaar en laden van algoritmes/modellen
* Databeheer: Algebra tools, statistieken, null-waarden, ...

Let op dat er twee API's aangeboden worden, 1 gebaseerd op RDD's en 1 op DataFrames.
De API gebaseerd op RDD's is ouder en minder flexibel dan de API gebruik makend van DataFrames.
Momenteel werken ze allebei maar in de toekomst zou de RDD gebaseerde kunnen verdwijnen.

## Utilities

### Varianten voor numpy-arrays

Voor feature sets en volledige matrices van datasets aan te maken kan je gebruik maken van de Vector en Matrix klassen.
Deze beschikken over een Dense variant waar je elk element moet ingeven of een Sparse Variant waar cellen, elementen leeg kan laten.
Dit ziet er als volgt uit:

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MLLib intro").getOrCreate()

Het is belangrijk om te weten dat dit locale datastructuren (wrapper rond numpy array) zijn en geen gedistribueerde objecten.

### Statistieken

Voor er kan gewerkt worden met statistieken moeten we (net zoals bij pandas) eerst een dataset hebben.
Hieronder maken we een random dataframe aan van 50 rijen en 4 kolommen.

**Correlation matrix**

Buiten de statistieken die berekend kunnen worden door de summary() functie kan ook de correlatiematrix belangrijk zijn.
Deze matrix maakt het mogelijk om het verband tussen de verscheidene features te bestuderen.
Deze matrix kan als volgt berekend worden voor een gedistribueerd dataframe.

**Onafhankelijksheidtest**

Naast de correlatiematrix kan het ook belangrijk zijn om de onafhankelijkheid te testen tussen elke feature en een label.
Dit kan uitgevoerd worden door een zogenaamde ChiSquareTest.
Deze krijgt als input een dataframe, de naam van de kolom met de features (als vectors) en de naam van een kolom met de labels.
We kunnen deze test uitvoeren als volgt:

De Chi-square test wordt op de volgende manieren gebruikt in Machine Learning:
* Feature selectie: Identificeer welke features een significante relatie hebben met de targetvariabele.
* Correlatie tussen categorische variabelen: Evalueer de afhankelijkheid tussen twee categorische variabelen.

De Chi-square test vergelijkt de verwachte en werkelijke frequenties van waarden in de categorieën van een feature ten opzichte van de targetvariabele. Het doel is om te bepalen of de verschillen tussen deze frequenties statistisch significant zijn of niet. Een hoge Chi-square score betekent dat er een grote afhankelijkheid is tussen de feature en de target, wat suggereert dat de feature nuttig is voor voorspellingen.
Als de p-waarde kleiner is dan een vooraf bepaald significantieniveau (bijvoorbeeld 0,05), verwerp je de nullhypothese (dat er geen relatie is), wat betekent dat de feature relevant is.

**Summarizer**

Andere statistieken per kolom kunnen berekend worden door gebruik te maken van de Summarizer klasse:

Het gebruik maken van de Summarizer maakt het dus mogelijk om rechtstreeks op de feature vectors te werken zonder ze eerst terug te moeten splitsen.

### Pipelines

Pipelines binnen Spark zijn een groep van high-level API's steunend op Dataframes om ML-pipelines aan te maken, optimaliseren en trainen.
De belangrijkste concepten binnen de Pipelines van Spark zijn:
* Dataframe: concept van de dataset
* Transformer: Zet een dataframe om in een ander dataframe
* Estimator: Zet een dataframe om in een model/transformer
* Pipeline: een ketting van transformers en estimators om een flow vast te leggen
* Parameter: API voor parameters van transformers en estimators aan te passen

Gebruik nu onderstaande mini-dataset waar we op basis van een tekstkolom met logistische regressie een bepaald label proberen te voorspellen.
Maak hiervoor een Pipeline uit die bestaat uit de volgende stappen:
* Tokenizer om de tekstkolom te splitsen in de overeenkomstige woorden
* HashingTf om de term frequency van de woorden te bepalen en het om te zetten naar een feature vector
* LogisticRegression Estimator om de voorspelling te doen.

Train daarna deze pipeline en maak de voorspellingen voor de traningsdata.
Hoe accuraat is dit model?

### Evalueren van een model

In de pyspark.ml package zitten er ook functionaliteiten voor deze modellen te evalueren.
Meer informatie hierover vind je [hier](https://spark.apache.org/docs/2.2.0/mllib-evaluation-metrics.html).

### Data sources

Door gebruik te maken van de sparkContext kunnen een reeks standaard databronnen ingelezen worden om datasets uit op te bouwen (Csv, Json, ...).
Daarnaast is het ook mogelijk om een folder met een reeks beelden te gebruiken als dataset om zo een model voor image classification te trainen.
Download nu [deze](https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin) dataset en upload ze naar een folder op het hadoop filesysteem.

In [ ]:
import opendatasets as od

od.download("https://www.kaggle.com/returnofsputnik/chihuahua-or-muffin")

In [ ]:
from hdfs import InsecureClient

client = InsecureClient('http://localhost:9870', user='bigdata')
map = "/user/bigdata/MLLib"

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    # do some cleaning in case anything else than *.txt is present
    for f in client.list(map):
        client.delete(map + '/' + f, recursive=True)

client.upload(map, 'chihuahua-or-muffin')

De geuploade images kunnen nu ingelezen worden als volgt:

Merk op dat het werken met images niet zo eenvoudig is.
Hiervoor wordt binnen pyspark typisch gebruik gemaakt van de [sparkdl](https://smurching.github.io/spark-deep-learning/site/api/python/sparkdl.html) package.
Hierbij staat de dl voor deep learning.
Aangezien dit ons momenteel te ver leidt ga ik dit niet verder toelichten.

Een andere aparte databron die eenvoudig ingelezen kan worden is het formaat "libsvm".
Een bestand van dit formaat wordt ingelezen als een dataframe met twee kolommen: een label en een kolom met de feature-vectors.
De code om dergelijk bestand in te laden is:

## Voorbeeld

Train en evalueer een machine learning model dat beeldgegevens kan classificeren, gebaseerd op eigenschappen zoals hoogte, breedte en aantal kanalen. Omdat MLlib geen directe ondersteuning biedt voor beelddata, zullen we enkele numerieke eigenschappen van de afbeeldingen gebruiken en pipelines opzetten voor preprocessing en training.
Hierbij ga je verder werken op het dataframe van de chichuahua of muffin dataset van hierboven.

Voer de volgende stappen uit:
* Data decoderen en omzetten naar een dataframe dat door ML-kan gebruikt worden.
    * Schrijf een udf om de image.data kolom om te zetten naar een vector. Gebruik hiervoor een udf dat dit eerst omzet naar een numpy array waarna het omgezet wordt naar een PIL-Image (een python package). Dit wordt dan geresized naar een 32x32 figuur. Ten slotte wordt dit omgezet naar een 1D-array 
    * Schrijf een udf om de filename in de image.origin kolom om te zetten naar 1 van de labels: 'muffin' of 'chihuahua'
* Na het uitvoeren van de udf's zou je een dataframe moeten hebben met minstens 2 kolommen: het label als string kolom, de data als een vector kolom
* Splits de data in een trainings en testset
* Voer de volgende preprocessing stappen uit
    * Zet de tekstuele labels om naar integers
    * Schaal de vector-kolom door elke waarde te delen door 255 zodat de pixel-waarden in de range 0-255 vallen
    * Zorg ervoor dat dit in een pipeline zit en fit de preprocessor al eens. Print het schema en een aantal rijen uit om de output te verifieren
* Voer data modelling uit
    * Zorg voor dimensionality reduction aan de hand van PCA
    * Gebruik logistische regressie om het label te voorspellen
* Evalueer het model door een confusion matrix te berekenen

## Oefening - classificatie

Voer de volgende stappen uit, volg de best-practices van het data science vak:
* Genereer een dataset met de make_classification functie van Scikit-Learn (of gebruik randomSplit op een bestaande dataset in PySpark).
* Converteer de dataset naar een PySpark DataFrame.
* Voer basisverkenning uit: toon het schema van de data, bekijk de eerste rijen, en controleer de kolomtypes.
* Gebruik VectorAssembler om alle features in één vector-kolom (features) te combineren.
* Controleer de structuur van de output.
* Train een Logistic Regression model op de gegenereerde dummy data.
* Gebruik een training/test split van 80/20.
* Evalueer het model door de nauwkeurigheid (accuracy) te berekenen.

## Oefening - hyperparameter tuning

Voer de volgende stappen uit, volg de best-practices van het data science vak:
* Genereer dummy data (of gebruik bestaande data) om een regressieprobleem op te lossen. (make_regression)
* Bouw een pipeline die de data voorbereidt door schaling uit te voeren, een RandomForest-model traint, en de beste hyperparameters selecteert. (Tip: ParamGridBuilder)
* Voer hyperparameter tuning uit en evalueer de prestaties van het beste model.